[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/d8ahazard/sd_dreambooth_extension/blob/Torch2/D8-DreamBooth.ipynb)

In [ ]:
from google.colab import drive
import os

drive.mount("/content/gdrive", force_remount=True)

if not os.path.exists("/content/gdrive/MyDrive/sd-db-d8"):
  os.mkdirs("/content/gdrive/MyDrive/sd-db-d8")

if not os.path.exists("/content/working/"):
  os.mkdirs("/content/working/")

!rm -f /content/working/stable-diffusion-webui
!ln -s /content/gdrive/MyDrive/sd-db-d8 /content/working/stable-diffusion-webui

Install various packages

In [ ]:
# **fast-DreamBooth colab From https://github.com/d8ahazard/sd_dreambooth_extension, modified from https://github.com/camenduru/stable-diffusion-webui-collab**
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/working/stable-diffusion-webui/
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/working/stable-diffusion-webui/scripts/run_n_times.py
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /content/working/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/working/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone https://github.com/camenduru/sd-civitai-browser /content/working/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone https://github.com/camenduru/sd-webui-additional-networks /content/working/stable-diffusion-webui/extensions/sd-webui-additional-networks
!git clone -b Torch2 https://github.com/d8ahazard/sd_dreambooth_extension /content/working/stable-diffusion-webui/extensions/sd_dreambooth_extension

%cd /content/working/stable-diffusion-webui


Bump Python Version (RUN ONCE)

In [ ]:

!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
!bash ./py310.sh -b -f -p /usr/local
!python -m ipykernel install --name "py310" --user


Download the 1.5 Model

In [ ]:
!wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.safetensors -O /content/working/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned.safetensors


And/Or download the 2.1 Model

In [ ]:
!wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-nonema-pruned.safetensors -O /content/working/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-nonema-pruned.safetensors

!wget https://huggingface.co/ckpt/stable-diffusion-2-1/raw/main/v2-inference-v.yaml -O /content/working/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-nonemaema-pruned.yaml


Install Dreambooth and Auto1111 dependencies (RUN ONCE)

In [ ]:
!%cd /content/working/stable-diffusion-webui/
# Clone Auto1111 repos
!git clone "https://github.com/Stability-AI/stablediffusion.git" "/content/working/stable-diffusion-webui/repositories/stable-diffusion-stability-ai" 
!%cd "/content/working/stable-diffusion-webui/repositories/stable-diffusion-stability-ai" 
!git checkout "47b6b607fdd31875c9279cd2f4f16b92e4ea958e"
!git clone "https://github.com/CompVis/taming-transformers.git" "/content/working/stable-diffusion-webui/repositories/taming-transformers" 
!%cd "/content/working/stable-diffusion-webui/repositories/taming-transformers" 
!git checkout "24268930bf1dce879235a7fddd0b2355b84d7ea6"
!git clone "https://github.com/crowsonkb/k-diffusion.git" "/content/working/stable-diffusion-webui/repositories/k-diffusion"
!%cd "/content/working/stable-diffusion-webui/repositories/k-diffusion"
!git checkout "5b3af030dd83e0297272d861c19477735d0317ec"
!git clone "https://github.com/sczhou/CodeFormer.git" "/content/working/stable-diffusion-webui/repositories/CodeFormer"
!%cd "/content/working/stable-diffusion-webui/repositories/CodeFormer" 
!git checkout "c5b4593074ba6214284d6acd5f1719b6c5d739af"
!git clone "https://github.com/salesforce/BLIP.git" "/content/working/stable-diffusion-webui/repositories/BLIP" 
!%cd "/content/working/stable-diffusion-webui/repositories/BLIP"
!git checkout "48211a1594f1321b00f14c9f7a5b4813144b2fb9"


Setup VENV

In [ ]:
# Install PIP stuff
!%cd /content/working/stable-diffusion-webui/
!pip install virtualenv
!virtualenv venv
!source venv/bin/activate
# Install Auto1111 PIP packages
!pip install git+https://github.com/TencentARC/GFPGAN.git@8d2447a2d918f8eba5a4a01463fd48e45126a379
!pip install git+https://github.com/openai/CLIP.git@d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
!pip install git+https://github.com/mlfoundations/open_clip.git@bb6e834e9c70d9c27d0dc3ecedeebeaeb1ffad6b

# Install Requirements
!pip install -r /content/working/stable-diffusion-webui/requirements_versions.txt
!pip install -r /content/working/stable-diffusion-webui./extensions/sd_dreambooth_extension/requirements.txt
!pip install https://github.com/ArrowM/xformers/releases/download/xformers-0.0.17%2B36e23c5.d20230209-cp310-cu118/xformers-0.0.17+36e23c5.d20230209-cp310-cp310-linux_x86_64.whl
!pip install https://download.pytorch.org/whl/nightly/cu118/torch-2.0.0.dev20230209%2Bcu118-cp310-cp310-linux_x86_64.whl https://download.pytorch.org/whl/nightly/cu118/torchvision-0.15.0.dev20230209%2Bcu118-cp310-cp310-linux_x86_64.whl


Run Auto1111

In [ ]:
!%cd /content/working/stable-diffusion-webui/extensions/sd_dreambooth_extension/
!git fetch && git pull
!%cd /content/working/stable-diffusion-webui/ 
!python launch.py --share --xformers --enable-insecure-extension-access --torch2 --skip-install --skip-torch-cuda-test